---
jupyter: python3
from: markdown+emoji
execute:
  echo: true
  eval: false
  message: false
  warning: false
---

# Réhaussement et visualisation d'images {#sec-chap02}

Assurez-vous de lire ce préambule avant d'exécutez le reste du notebook.

## :rocket: Préambule
### :dart: Objectifs
Dans ce chapitre, nous abordons quelques techniques de réhaussement et de visualisation d'images. Ce chapitre est aussi disponible sous la forme d'un notebook Python:

[![](images/colab-badge.svg)](https://colab.research.google.com/github/sfoucher/TraitementImagesPythonVol1/blob/main/notebooks/02-RehaussementVisualisationImages.ipynb){target="_blank"} 

### Librairies
Les librairies qui vont être explorées dans ce chapitre sont les suivantes:

* [SciPy -](https://scipy.org/)

* [NumPy -](https://numpy.org/) 

* [opencv-python · PyPI](https://pypi.org/project/opencv-python/)

* [scikit-image](https://scikit-image.org/)

* [Rasterio](https://rasterio.readthedocs.io/en/stable/)

* [Xarray](https://docs.xarray.dev/en/stable/)

* [rioxarray](https://corteva.github.io/rioxarray/stable/index.html)



Dans l'environnement Google Colab, seul `rioxarray` et GDAL doivent être installés:

In [ ]:
%%capture
!apt-get update
!apt-get install gdal-bin libgdal-dev
!pip install -q rioxarray
!pip install -qU "geemap[workshop]"

Vérifier les importations:

In [ ]:
#| eval: true
import numpy as np
import rioxarray as rxr
from scipy import signal
import xarray as xr
import xrscipy
import matplotlib.pyplot as plt

### Données
Nous allons utilisez les images suivantes dans ce chapitre:

In [ ]:
%%capture
!wget https://github.com/sfoucher/TraitementImagesPythonVol1/raw/refs/heads/main/data/chapitre01/subset_RGBNIR_of_S2A_MSIL2A_20240625T153941_N0510_R011_T18TYR_20240625T221903.tif -O RGBNIR_of_S2A.tif
!wget https://github.com/sfoucher/opengeos-data/raw/refs/heads/main/raster/landsat7.tif -O landsat7.tif
!wget https://github.com/sfoucher/opengeos-data/raw/refs/heads/main/images/berkeley.jpg -O berkeley.jpg
!wget https://github.com/sfoucher/TraitementImagesPythonVol1/raw/refs/heads/main/data/chapitre01/subset_0_of_S1A_split_NR_Cal_Deb_ML_Spk_SRGR.tif -O SAR.tif

Vérifiez que vous êtes capable de les lire :

In [ ]:
#| eval: true
#| output: false

with rxr.open_rasterio('berkeley.jpg', mask_and_scale= True) as img_rgb:
    print(img_rgb)
with rxr.open_rasterio('RGBNIR_of_S2A.tif', mask_and_scale= True) as img_rgbnir:
    print(img_rgbnir)
with rxr.open_rasterio('SAR.tif', mask_and_scale= True) as img_SAR:
    print(img_SAR)

## Réhaussements visuels

Le but du réhaussement visuel d'une image vise principalement à améliorer la qualité visuelle d'une image en améliorant le contraste, la dynamique ou la texture d'une image. De manière générale, ce réhaussement ne modifie pas la donnée d'origine mais est plutôt appliquée dynamiquement à l'affichage pour des fins d'inspection visuelle.

### Statistiques d'une image

On peut considérer un ensemble de statistique globales pour chacune des bandes d'une image:
- valeurs minimales et maximales
- valeurs moyennes, médianes et quantiles
- écart-types, skewness et kurtosis 
Ces statistiques doivent être calculées pour chaque bande d'une image multispectrale.

En ligne de commande, `gdalinfo` permet d'interroger rapidement un fichier image pour connaitre les statistiques de base:

In [ ]:
#| lst-label: lst-gdalstats
#| lst-cap: Statistiques d'une image avec gdal
#| eval: true

!gdalinfo -stats landsat7.tif

Les librairies de base comme `xarray` et `numpy` peuvent facilement produire des statistiques comme avec la fonction [stats](https://rasterio.readthedocs.io/en/stable/api/rasterio.io.html#rasterio.io.BufferedDatasetWriter.stats):

In [ ]:
#| eval: false

import rasterio as rio
import numpy as np
with rio.open('landsat7.tif') as src:
    stats= src.stats()
    print(stats)

La librairie `xarray` donne accès à des fonctionnalités plus sophistiquées comme le calcul des quantiles:

In [ ]:
#| eval: true

import rioxarray as riox
with riox.open_rasterio('landsat7.tif', masked= True) as src:
    print(src)
quantiles = src.quantile(dim=['x','y'], q=[.025,.25,.5,.75,.975])
quantiles

#### Calcul de l'histogramme

Le calcul d'un histogramme pour une image (une bande) permet d'avoir une vue plus détaillée de la répartition des valeurs radiométriques. Le calcul d'un histogramme nécessite minimalement de faire le choix d'une valeur du nombre de *bins* (ou de la largeur). Un *bin* est un intervalle de valeurs pour lequel on peut calculer le nombre de valeurs observées dans l'image. La fonction de base pour ce type de calcul est la fonction `numpy.histogram()`:

In [ ]:
#| eval: true
import numpy as np
array = np.random.randint(0,10,100) # 100 valeurs aléatoires entre 0 et 10
hist, bin_limites = np.histogram(array, density=True)
print('valeurs :',hist)
print(';imites :',bin_limites)

Le calcul se fait avec 10 intervalles par défaut.

Pour des besoins de visualisation, le calcul des valeurs extrêmes de l'histogramme peut aussi se faire via les quantiles comme discutés auparavant.

##### Visualisation des histogrammes

La librarie `rasterio` est probablement l'outil le plus simples pour visualiser rapidement des histogrammes sur une image multi-spectrale:

In [ ]:
#| eval: true
import rasterio as rio
from rasterio.plot import show_hist
with rio.open('RGBNIR_of_S2A.tif') as src:
  show_hist(src, bins=50, lw=0.0, stacked=False, alpha=0.3,histtype='stepfilled', title="Histogram")

### Réhaussements linéaires

Le réhaussement linéaire d'une image est la forme la plus simple de réhaussement, elle consiste 1) à optimiser les valeurs des pixels d'une image afin de maximiser la dynamique disponibles à l'affichage, ou 2) changer le format de stockage des valeurs (e.g. de 8 bit à 16 bit):

$$ \text{nouvelle valeur d'un pixel} = \frac{\text{valeur d'un pixel} - min_0}{max_0 - min_0}\times (max_1 - min_1)+min_1$${#eq-rehauss-lin}

Par cette opération, on passe de la dynamique de départ ($max_0 - min_0$) vers la dynamique cible ($max_1 - min_1$). Bien que cette opération semble triviale, il est important d'être conscient des trois contraintes suivantes:
1. **Faire attention à la dynamique cible**, ainsi, pour sauvegarder une image en format 8 bit, on utilisera alors $max_1=255$ et $min_1=0$.
2. **Préservation de la valeur de no data** : il faut faire attention à la valeur $min_1$ dans le cas d'une valeur présente pour *no_data*. Par exemple, si *no_data=0* alors il faut s'assurer que $min_1>0$.
3. **Précision du calcul** : si possible réaliser la division ci-dessus en format *float*

### Réhaussements non linéaires


Calcul d'histogrammes, étirement, égalisation, styling

### Composés couleurs

Le système visuel humain est sensible seulement à la partie visible du spectre électromagnétique qui compose les couleurs de l'arc-en-ciel du bleu au rouge. L'ensemble des couleurs du spectre visible peut être obtenu à partir du mélange de trois couleurs primaires (rouge, vert et bleu). Ce système de décomposition à trois couleurs est à la base de la plupart des systèmes de visualisation ou de représentation de l'information de couleur. On peut trouver des variantes comme le système HSV (*Hue-Saturation-Value*) utilisé en encodage de données vidéos.

## Visualisation

### Visualisation en Python

Il faut d'entrée mentionner que Python n'est pas vraiment fait pour visualiser de la donnée de grande taille, le niveau d'interactivité est aussi plus limité. Néanmoins, il est possible de visualiser de petites images avec la librairie Matplotlib. 

### Outils de visualisation

Il existe plusieurs outils gratuits de visualisation d'une image satellite, on peut mentionner les deux principaux:
- QGIS
- ESA Snap


### Visualisation sur le Web

Une des meilleures pratiques pour visualiser une image de grande taille est d'utiliser un service de type Web Mapping Service (WMS). Cependant, type de service nécessite une architecture client-serveur qui est plus complexe à mettre en place.

Google Earth Engine offre des moyens de visualiser de la donnée locale:
_Working with Local Geospatial Data_ — via [17. Geemap — Introduction to GIS Programming](https://geog-312.gishub.org/book/geospatial/geemap.html#working-with-local-geospatial-data)

via [data/raster at main · opengeos/data](https://github.com/opengeos/data/tree/main/raster)



### Visualisation 3D

drapper une image satellite sur un DEM


## Quiz de révision du chapitre

## Exercices de révision 
